# Gewählte xlsl Datei in csv Datei umwandeln und umbenennen

In [ ]:
# Schritt 1: Deinstalliere alle problematischen Pakete
!pip uninstall tensorflow tensorflow-text tf-keras tensorflow-decision-forests tensorflow-probability -y

# Schritt 2: Installiere kompatible Versionen
!pip install tensorflow==2.19.0 tensorflow-probability==0.25.0 tensorflow-text==2.19.0 tf-keras==2.19.0

# Schritt 3: Unterdrücke Warnungen (für parakeet und CUDA)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Unterdrückt TensorFlow-Warnungen

# Schritt 4: Teste den Import
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
print("Import erfolgreich!")

# Schritt 5: Teste tensorflow_probability
import tensorflow_probability as tfp
print(f"TensorFlow Probability Version: {tfp.__version__}")

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tensorflow-text 2.19.0
Uninstalling tensorflow-text-2.19.0:
  Successfully uninstalled tensorflow-text-2.19.0
Found existing installation: tf_keras 2.19.0
Uninstalling tf_keras-2.19.0:
  Successfully uninstalled tf_keras-2.19.0
Found existing installation: tensorflow-probability 0.25.0
Uninstalling tensorflow-probability-0.25.0:
  Successfully uninstalled tensorflow-probability-0.25.0
  Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorflow_probability-0.25.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached tensorflow_text-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64

In [6]:
import platform
import os
import pandas as pd
import random
import tensorflow_probability as tfp
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from google.colab import drive
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datetime import datetime
import shutil



In [2]:
def smartEncoding():
  plt = platform.system
  if plt == "Windows":
    return "utf-8-sig"
  else:
    return "utf-8"

In [3]:

read_file = pd.DataFrame(pd.read_excel('/content/drive/My Drive/Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).xlsx'))
read_file.to_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv", index = False, header = True, encoding=smartEncoding())
df = pd.DataFrame(pd.read_csv("Google_Rezensionen_qualitätsgesichert_2023_12_19 (1).csv"))
#print(df.head)

In [4]:
#wähle relevante Spalten
gewaehlte_spalten = df[["Erfahrungsbericht des Nutzers" , "Zufallszahl"]]

#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv", index = False)




# Datenbereinigung

In [8]:
erfahrungen_gefiltert = pd.read_csv("/content/drive/My Drive/erfahrungen_gefiltert.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_gefiltert.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_gefiltert.drop_duplicates(inplace=True)
#speichern
erfahrungen_gefiltert.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index=False)
print(f"FINAL: {len(erfahrungen_gefiltert)} Reihen")


#speichere sie in neue csv datei
gewaehlte_spalten.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)


#zufällig n = 100 Berichte für erfahrungen_final.csv auswählen
erfahrungen_clean = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")
# dropna() entfernt NaN-Reihen aus anderen Spalten
erfahrungen_clean.dropna(subset=['Erfahrungsbericht des Nutzers'], inplace=True)
# leere strings entfernen
erfahrungen_clean.drop_duplicates(inplace=True)
erfahrungen_clean.to_csv("/content/drive/My Drive/erfahrungen_clean.csv", index = False)

# INDEX-Nummern zufällig wählen
zufalls_index = random.sample(range(len(erfahrungen_clean)), k=100)
zufall_100 = erfahrungen_clean.iloc[zufalls_index]



#speichern (NUR 100!)
zufall_100.to_csv("/content/drive/My Drive/zufall_100_berichte.csv", index=False)

print(f"{len(zufall_100)} Zufallsberichte!")

FINAL: 6453 Reihen
100 Zufallsberichte!


# **Datenaugmentation und Syn-Chain-Methode**

## Datenaugmentation mit Trainingsdaten (ca. 11.900 Rezensionen)

für Fine Tuning:
Hugging Face: Fine-tune ein Grok-ähnliches Modell (z. B. "xai-org/grok-1" oder "reedmayhew/Grok-3-gemma3-4B-distilled" als Distillation von Grok 3). Das ist open-source und GitHub-freundlich.

Code-Beispiel (Distillation von Grok 3 via API + Fine-Tuning auf Gemma-3 4B):


In [ ]:
#!pip install transformers torch datasets accelerate huggingface_hub

#from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
#from datasets import Dataset
#import torch

# Lade und bereite Daten (aus vorherigem Code)
#data = pd.read_csv("/content/drive/My Drive/Rezensionen/erfahrungen_clean.csv")
#data['Label'] = data['Erfahrungsbericht des Nutzers'].apply(get_sentiment)
#data['Label'] = data['Label'].map({'positiv': 0, 'neutral': 1, 'negativ': 2})  # Numerisch

# Augmentierte Trainingsdaten (aus vorherigem Code, ca. 11.900 + augmentiert)
# train_data_augmented, test_data = ... (aus vorherigem Split)

#tokenizer = AutoTokenizer.from_pretrained('google/gemma-3-4b')  # Grok-ähnlich
#def tokenize(batch):
    #return tokenizer(batch['Erfahrungsbericht des Nutzers'], padding=True, truncation=True)

#train_dataset = Dataset.from_pandas(train_data_augmented).map(tokenize, batched=True)
#test_dataset = Dataset.from_pandas(test_data).map(tokenize, batched=True)

#model = AutoModelForSequenceClassification.from_pretrained('google/gemma-3-4b', num_labels=3)

#training_args = TrainingArguments(
    #output_dir='/content/drive/My Drive/Rezensionen/grok3_finetuned',
    #num_train_epochs=3,
    #per_device_train_batch_size=4,  # Passe an GPU an
    #evaluation_strategy='epoch',
    #save_strategy='epoch',
    #load_best_model_at_end=True,
    #push_to_hub=True  # Direkt zu HF Hub pushen (GitHub-ähnlich)
#)

#trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset, tokenizer=tokenizer)
#trainer.train()

# Push zu Hugging Face Hub (braucht HF-Token)
#trainer.push_to_hub("dein-username/grok3-mini-finetuned-sentiment", token="DEIN_HF_TOKEN")
#print("Modell auf Hugging Face Hub hochgeladen – kann zu GitHub-Repo verlinkt werden.")

Best Practices (aus Reddit, GitHub Blog, HF Docs):

Lizenz: Füge eine LICENSE (z. B. Apache 2.0) hinzu, da Grok-Modelle proprietär sind – dein Fine-Tuning erbt das.
Model Card: Erstelle eine (HF-Template: https://huggingface.co/docs/hub/model-cards) mit Details zu Daten (12.000 Rezensionen), Augmentation und Metriken.
Datenschutz: Anonymisiere sensible Daten in der CSV (z. B. Namen entfernen), bevor du hochlädst – Pflegeberichte könnten personenbezogen sein (DSGVO-konform!).
Versionskontrolle: Nutze GitHub Releases für große Dateien (z. B. Weights als Asset).
Integration mit HF: Push zu Hugging Face Hub und verlinke im GitHub-Repo – HF ist GitHub-ähnlich und unterstützt Grok-Destillationen (z. B. "reedmayhew/Grok-3-gemma3-4B-distilled").



3. Empfehlung für deine Bachelorarbeit

Starte mit Option A/B: Prompt Engineering für schnelle Tests, dann Hugging Face für echtes Fine-Tuning (ca. 30–60 Min. in Colab mit GPU).
GitHub-Repo als Portfolio: Lade Code, Skripte und Metriken hoch – das zeigt Reproduzierbarkeit. Vermeide rohe Weights (>100 MB) direkt; verlinke sie.
Nächste Schritte: Hole dir einen xAI API-Key und HF-Token. Teste den Code in Colab.

Falls du Hilfe beim Setup (z. B. API-Key, spezifischer Code-Fehler) brauchst oder mehr Details zu einer Option, lass es mich wissen!

## Paraphrasieren

In [8]:
#csv Datei laden
data = pd.read_csv("/content/drive/My Drive/erfahrungen_clean.csv")

# Erstellung Pegasus-Modell für Paraphrasing (kompatibel mit Colab)
model_name = "google/pegasus-x-base"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def paraphrase_text(text):
  try:
    inputs = tokenizer(f"paraphrase: {text}", max_length=128, truncation = True, return_tensors = "pt")
    outputs = model.generate(
        **inputs,
        max_length = 128,
        num_beams = 5,
        early_stopping = True
    )
    paraphrased_text = tokenizer.decode(outputs[0], skip_special_tokens = True)
    return paraphrased_text
  except Exception as e:
    print(f"Fehler beim Paraphrase für folgenden Text: {text}[:50]...- {e}")
    return text
#Paraphrasieren von Trainingsdaten (Demo: 10, für 12.000 für GPU Colab Pro)
paraphrased_texts = []
for idx, row in data.head(10).iterrows():#ersetze head(an Stelle 10) mit data
  text = row['Erfahrungsbericht des Nutzers']
  zufallszahl = row['Zufallszahl']
  paraphrased_text = paraphrase_text(text)
  paraphrased_texts.append({
        'Original': text,
        'Paraphrase': paraphrased_text,
        'Zufallszahl': zufallszahl
    })

# In DataFrame speichern
paraphrased_df = pd.DataFrame(paraphrased_texts)

# Speichere in Google Drive
ordner = "/content/drive/My Drive/Rezensionen/"
os.makedirs(ordner, exist_ok=True)
alter_pfad = ordner + "paraphrased_pegasus_temp.csv"
paraphrased_df.to_csv(alter_pfad, index=False)
neuer_pfad = ordner + f"paraphrased_pegasus_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
os.rename(alter_pfad, neuer_pfad)
print(f"Paraphrasierte Daten gespeichert als '{neuer_pfad}'.")


#Paraphrasing und Grammatical Variation
#Synonymersetzung (nlpaug) & generative Paraphrasierung (german-t5-efficient).




You are using a model of type pegasus_x to instantiate a model of type pegasus. This is not supported for all configurations of models and can yield errors.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-x-base and are newly initialized: ['model.decoder.embed_positions.weight', 'model.decoder.layers.0.encoder_attn.k_proj.bias', 'model.decoder.layers.0.encoder_attn.out_proj.bias', 'model.decoder.layers.0.encoder_attn.q_proj.bias', 'model.decoder.layers.0.encoder_attn.v_proj.bias', 'model.decoder.layers.0.self_attn.k_proj.bias', 'model.decoder.layers.0.self_attn.out_proj.bias', 'model.decoder.layers.0.self_attn.q_proj.bias', 'model.decoder.layers.0.self_attn.v_proj.bias', 'model.decoder.layers.1.encoder_attn.k_proj.bias', 'model.decoder.layers.1.encoder_attn.out_proj.bias', 'model.decoder.layers.1.encoder_attn.q_proj.bias', 'model.decoder.layers.1.encoder_attn.v_proj.bias', 'model.decoder.layers.1.self_attn.k_proj.bias', '

Paraphrasierte Daten gespeichert als '/content/drive/My Drive/Rezensionen/paraphrased_pegasus_20251022_224002.csv'.


In [ ]:
# um damit zu beginnen: https://arxiv.org/html/2507.09485